# 1. Title: Construction of an inference model for a multiple linear regression
Members:
- Mark Cao 
- Aarush Agarwal
- Edward Kim
- Ibrahim Saeed
  
Course: STAT301 102

April 16th, 2025


# Library

In [1]:
# Main Developer: Mark

library(tidyverse)
library(repr)
library(broom)
library(leaps)
library(moderndive)
library(MASS)
library(car)
library(rsample)
library(ggplot2)

print("LIBRARIES LOADED")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




[1] "LIBRARIES LOADED"


# 2. Introduction

Sleep plays a crucial role in maintaining physical and mental health. Disruptions in sleep quality or duration have been consistently associated with increased stress levels, cardiovascular risk, and metabolic issues such as obesity. As lifestyles become more demanding and sedentary, understanding how daily habits impact sleep health is both timely and essential. For this project, we use the Sleep Health and Lifestyle Dataset which contains 374 observations and 13 variables, covering sleep quality, daily activity, stress, and cardiovascular health. It includes numerical and categorical data, such as sleep duration (hours), self-reported sleep quality (1-10 scale), physical activity level (minutes/day), stress level (1-10 scale), BMI category, and presence of sleep disorders (None, Insomnia, Sleep Apnea). This dataset helps analyze the relationship between lifestyle choices and sleep health. 

Our inference-based research question is as follows:
We wish to examine the association between a person's sleep duration and health-routine-related predictors such as BMI, physical activity level, sleep disorder, stress level, etc.
Our question is primarily focused on inference, as our goal is to examine association intead of predicting values. By examining the model’s coefficients, we can interpret how each factor contributes to changes in sleep duration, providing insights into potential lifestyle adjustments for better sleep health.

There’s already quite a bit of research highlighting how lifestyle choices are connected to sleep health. For example, Grandner (2017) points out that behaviors like physical activity and diet play an important role in sleep quality. Medic et al. (2017) explain how poor sleep can affect both mental and physical health, especially through stress and cardiovascular issues. Similarly, Khan and Aouad (2017) show links between sleep loss and cardiovascular risk factors like high BMI and blood pressure. These studies help back up the relevance of the variables we’re exploring and show why it makes sense to look for associations between them and sleep disorders in this project.

# 3. Methods and Results

### a) Data 

In [2]:
<<<<<<< LOCAL CELL DELETED >>>>>>>
# Import required libraries
library(tidyverse)
library(repr)
library(broom)
library(leaps)
library(moderndive)
library(MASS)
library(car)
library(rsample)
library(ggplot2)

print("LIBRARIES LOADED")

ERROR: Error in parse(text = x, srcfile = src): <text>:1:1: unexpected input
1: <<
    ^


In [ ]:
# Main Developer: Mark

file_url <- "https://drive.google.com/uc?export=download&id=1ZjZvLl5dUzHEF8ouimlTg8t0MorhjzVA"
sleep_data <- read.csv(file_url)
head(sleep_data)
set.seed(114514) # SEED, DO NOT CHANGE

Dataset used: Laksika Tharmalingam. 2024. Sleep Health and Lifestyle Dataset. Retrieved from https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset/data

A detailed description of all the variables in the dataset is as follows:-

| Variable | Type of Variable | Description |
| -------- | --------------- | ----------- |
| Person ID | Numeric | Unique identifier for each individual |
| Gender | Categorical | Gender of the individual (Male/Female) |
| Age | Numeric | Age of the individual in years |
| Occupation | Categorical | Job or profession of the individual |
| Sleep Duration | Numeric | Number of hours a person sleep per day |
| Quality of Sleep | Numeric | Subjective sleep quality rating (1-10) |
| Physical Activity Level | Numeric | Minutes of physical activity of an individual per day |
| Stress Level | Numeric | Subjective stress level of a person (1-10) |
| BMI Category | Categorical | BMI classification (Underweight, Normal, Overweight) |
| Blood Pressure  | Numeric | The blood pressure measurement of the person, indicated as systolic pressure over diastolic pressure. |
| Heart Rate | Numeric | Resting heart rate in beats per minute |
| Daily Steps | Numeric | Number of steps taken per day |
| Sleep Disorder | Categorical | Presence of a sleep disorder (None, Insomnia, Sleep Apnea) |

#### The source of the dataset states that the set is synthetic and was generated for illustrative purposes.

### b) Exploratory Data Analysis (EDA)

#### Tidying and pre-selection

Tidying the data for modelling:
-  We remove the PersonID column, as it provides no statistical information.
-  For readability, we replace all column names, categorical level names, and other string data with versions without periods or whitespace in between characters.
-  For readability we replace these column names: "QualityofSleep" -> "SleepQuality", and "PysicalActivityLevel" -> "PhysicalActivity".
-  We perform factor conversion to all categorical variables.
- We replace the "BloodPressure" column with two new columns "SystolicPressure" and "DiastolicPressure" to maintain proper numerical columns for modelling.
- For simplicity we replace all "Obese" entries in BMICategory with "Overweight" so that only two levels remain: "Normal" and "Overweight"
- For simplicity we replace all SleepDisorder entries with either "None" for no sleep disorder and "Present" for any kind of sleep disorder.
- For future interpretability we replace the DailySteps column with Daily1000Steps for the number of daily steps of an individual divided by 1000.
- For future interpretability we divide all entries in PhysicalActivity by 60 to represent daily physical activity level of an individual in hours instead of minutes.
  
We set the reference level for each categorical variable as the following:
- BMICategory: "Normal"
- SleepDisorder: "None"
- Gender: "Male"
- Occupation: "Accountant"

In terms of pre-selection, the only complete removal was the PersonID column.


In [ ]:
sleep_data_reduced <- sleep_data |>
dplyr::select(-`Person.ID`) %>%
{ names(.) <- names(.) |> str_replace_all("[\\.\\s]", "")
.} %>%
mutate(Occupation = str_replace_all(Occupation, "[\\.\\s]", "")) |>
mutate(PhysicalActivityLevel = PhysicalActivityLevel / 60) |>
separate(BloodPressure, into = c("SystolicPressure", "DiastolicPressure"), sep = "/") |>
mutate(across(c(SystolicPressure, DiastolicPressure), as.numeric)) |>
mutate(BMICategory = if_else(str_detect(BMICategory, "Obese"), "Overweight", BMICategory)) |>
mutate(SleepDisorder = if_else(SleepDisorder == "None", "None", "Present")) |>
mutate(Daily1000Steps = DailySteps / 1000) |>
dplyr::select(-DailySteps) |>
rename(SleepQuality = QualityofSleep, PhysicalActivity = PhysicalActivityLevel) |>
mutate(
    BMICategory = fct_relevel(BMICategory, "Normal"),
    SleepDisorder = fct_relevel(SleepDisorder, "None"),
    Gender = fct_relevel(Gender, "Male"),
    Occupation = fct_relevel(Occupation, "Accountant")
  ) |>
mutate(across(where(is.character), as_factor))

head(sleep_data_reduced) 
nrow(sleep_data_reduced)

We note again that the new tidy and pre-selected dataset contains:

- "SleepQuality" name change from "QualityofSleep"
- "PhysicalActivity" as HOURS of physical activity per day instead of minutes
- "BMICategory" as either normal or overweight
- "SystolicPressure" from the systolic part of the blood pressure column
- "DiastolicPressure" from the diastolic part of the blood pressure column
- "SleepDisorder" as either None or Present
- "Daily1000Steps" for steps per day divided by 1000

In [ ]:
# Main Developer: Aarush

# VISUALIZING RESPONSE VARIABLE WITH INPUT VARIABLES
plot <- ggplot(sleep_data_reduced, aes(x = StressLevel, y = SleepDuration, color = PhysicalActivity)) +
  geom_point(size = 3, alpha = 0.7) +                                                 
  labs(title = "Exploring impact of Stress, Physical Activity, and BMI on Sleep Duration",
       x = "Stress Level",
       y = "Sleep Duration (hours)",
       color = "Physical Activity Level") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.1))

plot

From this scatterplot, we can see higher stress levels are associated with shorter sleep duration. People with more physical activity tend to sleep longer, suggesting that physical activity may help buffer the negative effects of stress on sleep.

In [ ]:
# Main Developer: Aarush
# PLOTTING HEATMAP
num_vars <- sleep_data_reduced |>
  dplyr::select(where(is.numeric))
cor_matrix <- round(cor(num_vars, use = "complete.obs"), 2)

melted_cor <- melt(cor_matrix)
ggplot(data = melted_cor, aes(x = Var1, y = Var2, fill = value)) +
  geom_tile(color = "white") +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", 
                       midpoint = 0, limit = c(-1, 1), name = "Correlation") +
  geom_text(aes(label = value), color = "black", size = 3) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1)) +
  labs(title = "Correlation Between Numeric Variables",
       x = "", y = "")

In this heatmap, we can see the pairwise correlations between numeric variables. We observe strong negative correlations between Sleep Duration, Sleep Quality, and Stress Level, indicating that higher stress is associated with poorer sleep outcomes. A very strong positive correlation is seen between Systolic and Diastolic Pressure, suggesting potential multicollinearity if both are used in modeling. This visualization helps identify variable relationships and warns of redundant predictors.

In [ ]:
sum(is.na(sleep_data_reduced))

We can see that there are no NAs in the dataset

In [ ]:
# Main Developer: Aarush

sleep_data_reduced %>%
  count(BMICategory, SleepDisorder, Gender) %>%
  arrange(desc(n))

This table shows how participants are grouped by BMI, sleep disorder status, and gender. It helps us check data balance and spot issues like rare categories (e.g., "Normal Weight"), which may need fixing before modeling.

### c) Methods: Plan

INFERENCE QUESTION: TO SEE IF THERE IS AN ASSOCIATION BETWEEN RESPONSE VARIABLE SLEEP DURATION AND OTHER HEALTH AND ROUTINE FACTORS SUCH AS PHYSICAL ACTIVITY, OCCUPATION, AGE, STRESS LEVEL.

### Assumptions:
- Linear relationship between Sleep duration(response) and the person's physical and mental health(input)
- Errors are independent
- Conditional distribution of the error term is normal
- Variances of the error term are equal
- Furthermore, I will use variable selection methods to minimize the effect of multicollinearity.
### Potential drawbacks and limitations:
- Even though sleep duration is a continuous variable, the relationship might not be perfectly linear
- Potential issues with our variable selection step.

### Data splitting

We first split the data 50%/50% for selection/inference:

In [ ]:
# SPLITTING
data_split <- sleep_data_reduced |> initial_split(prop = 0.5, strata = SleepDuration)
sleep_selection <- training(data_split)
sleep_inference <- testing(data_split)
print(paste("Selection N-ROWS:", nrow(sleep_selection)))
print(paste("Inference N-ROWS:", nrow(sleep_inference)))

### Initial VIF check
First we do a quick VIF check on the full model using the selection set:

In [ ]:
# FIRST VIF CHECK WITH FULL DATA
sleep_full <- lm(SleepDuration~., data=sleep_data_reduced)
vif(sleep_full)

### Variables with high scaled-GVIF:

SleepQuality, SystolicPressure, DiastolicPressure.

### VIF-based selection

Here we choose to manually remove the "DiastolicPressure" column from the model:

- Due to multicollinearity with SystolicPressure
- To simply the model and increase performance while keeping a similar representation to having both blood pressure values

In [ ]:
# MANUALLY REMOVE PROBLEMATIC DIASTOLIC PRESSURE COLUMN
sleep_step_ready <- sleep_data_reduced |>
  dplyr::select(-DiastolicPressure)

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
# Main Developer: Edward
# SPLITTING
data_split <- sleep_step_ready |> initial_split(prop = 0.7, strata = SleepDuration)
sleep_train <- training(data_split)
sleep_test <- testing(data_split)
print(paste("TRAINING N-ROWS:", nrow(sleep_train)))
print(paste("TESTING N-ROWS:", nrow(sleep_test)))

#### Model selection

Here we refit the full model and then use the selection set to perform model selection with stepAIC:

In [ ]:
# RE-FIT FULL MODEL WITH TRAINING SET WITH PROBLEMATIC COLUMN REMOVED
sleep_full <- lm(SleepDuration~., data=sleep_train)
# STEP AIC WITH FULL MODEL FROM TRAINING SET
aic_model <- stepAIC(sleep_full, direction = "backward", k = log(nrow(sleep_train)))

In [ ]:
full_vars <- attr(terms(sleep_full),"term.labels")
final_vars <- attr(terms(aic_model),"term.labels")
dropped_vars <- setdiff(full_vars, final_vars)
print(dropped_vars)

### Interpretation:

These variables were dropped after stepAIC: Age, StressLevel, HeartRate, SleepDisorder.



#### Results

Here we run tidy() on the new lm() with stepAIC-selected columns and the inference set to obtain the final result and p values:

In [ ]:
inference_results <- tidy(
  lm(formula = SleepDuration ~ Gender + Occupation + SleepQuality + 
      PhysicalActivity + BMICategory + SystolicPressure + Daily1000Steps, 
      data = sleep_inference)
)

print("1.")
print("GENERAL RESULTS OF THE FINAL MODEL:")
print(inference_results)

inference_results$direction <- ifelse(
  inference_results$p.value < 0.05,
  ifelse(inference_results$estimate > 0, "Positive", "Negative"), "Neither"
)

# Split into two groups
occupation_vars <- inference_results[grepl("^Occupation", inference_results$term), ]
other_vars <- inference_results[
  !grepl("^Occupation|\\(Intercept\\)", inference_results$term),
]

# Print categorized results
print("2.")
print("OCCUPATION LEVELS (relative to reference: Accountant):")
print(occupation_vars)

print("3.")
print("ALL OTHER PREDICTORS:")
print(other_vars)


In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
# Main Developer: Edward
model_results <- tidy(aic_model)
print("1.")
print("GENERAL RESULTS OF THE FINAL MODEL:")
print(model_results)
model_results$direction <- ifelse(
  model_results$p.value < 0.05,
  ifelse(model_results$estimate > 0, "Positive", "Negative"),"Neither")
occupation_vars <- model_results[grepl("^Occupation", model_results$term), ]
sleep_disorder_var <- model_results[grepl("^SleepDisorder", model_results$term), ]
non_occupation_vars <- model_results[!grepl("^Occupation|^SleepDisorder|\\(Intercept\\)", model_results$term), ]
print("2.")
print("OCCUPATION LEVELS (relative to reference: Accountant):")
print(occupation_vars)
print("3.")
print("SLEEP DISORDER VARIABLE (relative to reference: None):")
print(sleep_disorder_var)
print("4.")
print("ALL OTHER NUMERICAL VARIABLES:")
print(non_occupation_vars)

Interpretation:
From these stepAIC-selected inference-set model results, we conclude that the chosen factors indeed have a significant association with the response variable sleep duration.

We use the common threshold of p<0.05 to assess significance.

NOTE: THE SIGNIFICANCE VALUES FOR THE LEVELS OF THE CATEGORICAL VARIABLE OCCUPATION ARE PRODUCED IN ASSOCIATION WITH THE REFERENCE LEVEL "ACCOUNTANT", AND THE VALUES FOR THE OTHER NUMERICAL VARIABLES ARE PRODUCED IN ASSOCIATION WITH THE RESPONSE VARIABLE.

Namely:

- Higher values for SleepQuality and PhysicalActivity were found to be associated with longer sleep durations.
- Higher values for SystolicPressure and Daily1000Steps were found to be associated with shorter sleep durations.

Gender:

- Being female as opposed to male (reference level) showed association with longer sleep durations.

BMICategory:

- The "Overweight" category was not significantly different from the "Normal" reference group.

Sleep Disorder:

- This variable was not included in the final model.

As for Occupations:

During the tidying process, the reference level for Occupation was set to "Accountant". Therefore we expect "OccupationAccountant" to be absorbed into the reference level (Intercept) row of the stepAIC summary.

- Compared to accountants, managers did not show a significant difference in sleep duration (p-value ≥ 0.05).
- All other occupations showed a higher sleep duration compared to accountants, and these differences were statistically significant.

In terms of inference, the only solid conclusion we can make is that overall, occupation, along with gender, sleep quality, physical activity, systolic pressure, and daily steps, does have a significant association with sleep duration.

### Second VIF check
Here, we run a 2nd VIF check just for a quick comparison:

In [ ]:
# 2ND VIF WITH AIC MODEL
vif(lm(formula = SleepDuration ~ Gender + Occupation + SleepQuality + 
      PhysicalActivity + BMICategory + SystolicPressure + Daily1000Steps, 
      data = sleep_inference))

### Interpretation: 
A big improvement!

In [ ]:
<<<<<<< REMOTE CELL DELETED >>>>>>>
# Main Developer: Edward
# 2ND VIF WITH AIC MODEL
vif(aic_model)

#### Results Visualization
Here we use a visualization of the confidence intervals and means of the estimates, separated by variable:

In [ ]:
model_results <- tidy(aic_model, conf.int = TRUE) |>
  filter(term != "(Intercept)") |>
  mutate(direction = ifelse(p.value < 0.05,ifelse(estimate>0, "Positive", "Negative"), "Neither"))

occupation_vars <- model_results |> filter(grepl("^Occupation", term))
sleep_disorder_vars <- model_results |> filter(grepl("^SleepDisorder", term))
other_vars <- model_results |>
  filter(!grepl("^Occupation", term) & !grepl("^SleepDisorder", term))

plot_ci <- function(data, title) {
  ggplot(data, aes(x = estimate, y = reorder(term, estimate))) +
    geom_point() +
    geom_errorbarh(aes(xmin = conf.low, xmax = conf.high), height = 0.2) +
    geom_vline(xintercept = 0, linetype = "dashed", color = "red") +
    labs(title = title, x = "Estimate", y = "Predictor")
}

plot_ci(occupation_vars, "Occupation Type Estimates vs. 'Accountant', with 95% CI")

### Interpretation:

These graphs have a clean visualization of the significance of each predictor using mean and CI, as well as showing the direction of association, all corresponding to results we have explained above.


# 4. Discussion

Discussion and Conclusion
Our analysis explored the relationships between sleep duration and a range of predictors using a linear regression model applied to the Sleep Health and Lifestyle Dataset. The results provide valuable insights into the factors associated with sleep duration, while also highlighting the complexity of sleep health.
Key Findings
The model identified several significant associations. SleepQuality and PhysicalActivity were positively linked to sleep duration, suggesting that better sleep quality and higher physical activity levels contribute to longer sleep. In contrast, StressLevel and SystolicPressure showed negative associations, indicating that elevated stress and blood pressure are linked to shorter sleep, aligning with Khan and Aouad (2017) on stress and cardiovascular impacts on sleep.
Age had a modest positive association, implying that older individuals tend to sleep longer, very likely due to age-related changes in lifestyle or sleep needs. The negative association with Daily1000Steps was unexpected, as it is also a form of physical activity,  we suspect this unlikely result to be due to multicollinearity, although not handled by stepAIC. Occupation also played a significant role. Professions like OccupationDoctor , OccupationSalesRepresentative , and OccupationLawyer were associated with longer sleep duration compared to the reference group, potentially due to differences in work schedules or socioeconomic factors. However, OccupationManager showed no significant effect, indicating that occupational influences on sleep are not uniform.
Interestingly, SleepDisorderPresent was positively associated with sleep duration, a counterintuitive finding. This could reflect compensatory behaviors, such as spending more time in bed to offset poor sleep quality, or it may point to a limitation in the data or model specification.
The model’s adjusted R-squared of 0.9043 indicates that it explains a substantial portion of the variability in sleep duration, with a residual standard error of 0.2484 suggesting a good fit. However, residuals ranging from -0.8889 to 0.9432 hint at some unaccounted variability, possibly due to omitted factors like diet or mental health.
Limitations
Several limitations temper these findings. 
While associations are clear, we cannot determine whether predictors like stress or physical activity directly influence sleep duration. 
The sample size (374 observations) is relatively small, potentially limiting the generalizability of results, especially for less-represented occupations or conditions like sleep disorders.
 Also, the fact that this dataset is generated might make it not so representative of the actual population. 
Self-reported measures (if we consider the dataset as a result of random survey), such as sleep quality and stress level, may introduce bias due to subjectivity. 
Lastly, the unexpected positive association with sleep disorders gives suspicious results, as it could stem from unmeasured confounders or data artifacts.
Implications and Future Directions
These findings underscore the multifaceted nature of sleep health. The positive roles of sleep quality and physical activity suggest that interventions promoting exercise and sleep hygiene could enhance sleep duration. Conversely, the negative effects of stress and systolic pressure highlight the need for stress management and cardiovascular health strategies in sleep interventions. Occupation-specific differences suggest that workplace wellness programs could be tailored to support sleep health, particularly in high-stress roles.
In conclusion, this analysis reveals significant associations between sleep duration and a range of lifestyle and health-related factors. While the model offers robust explanatory power, its cross-sectional design and reliance on self-reported data call for cautious interpretation. Future research could leverage longitudinal data, objective measures (e.g., actigraphy for sleep, wearable devices for activity), and additional variables (e.g., diet, mental health) to strengthen these findings. Exploring interactions between factors, such as stress and physical activity, could further illuminate their combined effects on sleep. These steps would enhance our understanding of sleep health and inform targeted strategies to improve well-being.




We used stepAIC to find the best model, it works by adding or removing variables based on AIC. With Step AIC, we were able to drop 3 variables: Gender, 'BMI Catgeory', and 'HeartRate'. This gave us a good final model that balances complexity and interpretability.



